In [2]:
import yfinance as yf
from datetime import date
import pandas as pd
import numpy as np
data = yf.Ticker("MSFT")
data.history()
date.today()
date.today().strftime("%Y-%m-%d")


'2021-11-26'

In [3]:
tracking_stocks = {}

class stockData:
    def __init__(self, ticker):
        self.ticker = ticker
        data = self.connection()
        self.current_price = self.getCurrentPrice(data)
        self.historic_data = self.getHistoricPrice(data)

    def connection(self):
        return yf.Ticker(self.ticker)

    def getCurrentPrice(self, data = None):
        if data == None:
            data = self.connection()
        return data.info["currentPrice"]

    def getHistoricPrice(self, data = None):
        if data == None:
            data = self.connection()
        return data.history(period = "Max")


class portfolioData:
    def __init__(self, stocks_input = [], weights = [], description = ""):
        self.description = description
        self.stocks = {}
        self.weights = {}
        for i, stock in enumerate(stocks_input):
            weight = weights[i]/sum(weights)
            if stock not in tracking_stocks.keys():
                tracking_stocks[stock] = stockData(stock)
            self.weights[stock] = weight
            self.stocks[stock] = tracking_stocks[stock]
        self.updateWeights
            
    def updateAll(self):
        for stock in self.stocks.keys():
            self.stocks[stock].current_price = self.stocks[stock].getCurrentPrice()
            
    def listAllPrices(self):
        for stock in self.stocks.keys():
            print(stock, ": ", self.stocks[stock].current_price, sep = "")
    
    def listAllWeights(self):
        for stock in self.weights.keys():
            print(stock, ": ", self.weights[stock], sep = "")
            
    def addStock(self, stock, weight):
        adjustable_weight = (1 - weight)
        if stock in self.weights.keys() and len(self.weights.keys()) > 1:
            adjustable_weight = (1 - weight)/(1 - self.weights[stock])
        self.updateWeights(weight = adjustable_weight)
        if stock not in tracking_stocks.keys():
            tracking_stocks[stock] = stockData(stock)
        self.stocks[stock] = tracking_stocks[stock]
        self.weights[stock] = weight
        self.checkWeights()
    
    def updateWeights(self, weight):
        for stock in self.weights.keys(): 
            new_weight = self.weights[stock]*(weight)
            self.weights[stock] = new_weight
    
    def checkWeights(self):
        total_weight = 0
        for stock in self.weights.keys():
            total_weight += self.weights[stock]
        if total_weight != 1:
            adjustment = 1/total_weight
            self.updateWeights(adjustment)
    
    def historicPortfolio(self):
        stock_prices = None
        for stock in self.stocks.keys():
            if stock_prices is None:
                stock_prices = self.stocks[stock].historic_data["Close"]
                stock_prices = stock_prices.rename(stock)
                weights = pd.DataFrame({stock:np.repeat(self.weights[stock], len(stock_prices))}, index=stock_prices.index)
            else:
                temp = self.stocks[stock].historic_data["Close"]
                stock_prices = pd.merge(stock_prices, temp, how = "outer", 
                              left_on = "Date", right_on = "Date")
                stock_prices = stock_prices.rename(columns = {"Close":stock})
                temp_weight = pd.DataFrame({stock:self.weights[stock]}, index = temp.index)
                weights = pd.merge(weights, temp_weight, left_index = True, right_index = True, how = "outer")
        stock_prices = stock_prices.reset_index().sort_values("Date").reset_index(drop = True)
        weights = weights.reset_index().sort_values("Date").reset_index(drop = True)
        adjustments = weights.drop("Date", axis = 1).sum(axis = 1)
        weights = weights.iloc[:,1:].div(adjustments, axis = 0)
        return [stock_prices,weights]

In [4]:
def front_end():
    portfolios = {}
    

def create_portfolio():
    print("What would you like to call the portfolio?")
    name = input()
    running = True
    stocks = []
    weights = []
    while running:
        print("Add a stock ticker to the portfolio (leave blank to exit):")
        ticker = input()
        if ticker == "":
            running = False
            break
        stocks.append(ticker)
        weight = add_weight(ticker)
        weights.append(weight)
    checked_stocks = []
    checked_weights = []
    for i, ticker in enumerate(stocks):
        data = yf.Ticker(ticker)
        if data.info["regularMarketPrice"] != None:
            checked_stocks.append(ticker)
            checked_weights.append(weights[i])
    checked_weights = list(np.array(checked_weights)/sum(checked_weights))
    return [checked_stocks, checked_weights]

def add_weight(ticker):
    print(f"Add a weight to {ticker}:")
    weight = input()
    try:
        weight = float(weight)
    except:
        print("Please define weight on the format scale from 0-1.")
        add_weight(ticker)
    return weight
    
            
create_portfolio()       
        

What would you like to call the portfolio?
MSFT
Add a stock ticker to the portfolio (leave blank to exit):
0.5
Add a weight to 0.5:
0.5
Add a stock ticker to the portfolio (leave blank to exit):
GOOGL
Add a weight to GOOGL:
1
Add a stock ticker to the portfolio (leave blank to exit):



[['GOOGL'], [1.0]]